In [4]:
import pandas as pd

print(pd.__version__)


2.3.2


In [3]:
movies = pd.read_csv("data/movies_metadata.csv", low_memory=False)
ratings = pd.read_csv("data/ratings.csv")




In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kazia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kazia\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kazia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(tag):
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV
    }
    return tag_dict.get(tag[0].upper(), wordnet.NOUN)

def lemmatize_sentence(sentence):
    tokens = word_tokenize(sentence)        # tokenize sentence
    pos_tags = pos_tag(tokens)              # part-of-speech tagging
    return " ".join([lemmatizer.lemmatize(word, get_wordnet_pos(pos)) 
                     for word, pos in pos_tags])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer  # Fixed: 'fasture'->'feature', 'TfIdFVectorizer'->'TfidfVectorizer'
from nltk import word_tokenize, pos_tag  # Fixed: 'top_tag'->'pos_tag'
from nltk.stem.wordnet import WordNetLemmatizer  # Fixed: 'nordnet'->'wordnet', 'NordNetImmersion'->'WordNetLemmatizer'
from sklearn.metrics.pairwise import cosine_similarity  # Fixed: 'palvoids'->'pairwise', 'cosine_dimilarity'->'cosine_similarity
import pickle  # Fixed: 'picks'->'pickle' (this was correct)

In [6]:
movies = pd.read_csv('movies_metadata.csv')
ratings = pd.read_csv('ratings_small.csv')   # or ratings.csv depending on which one you want
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')
links = pd.read_csv('links.csv')
links_small = pd.read_csv('links_small.csv')


C:\Users\kazia\AppData\Local\Temp\ipykernel_24224\40784805.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('movies_metadata.csv')


In [ ]:
genre.info()

In [ ]:
movie.columns

In [7]:
import ast

# The Kaggle movies_metadata.csv already has genres
movies['genres'] = movies['genres'].fillna('[]').apply(ast.literal_eval)
movies['genres'] = movies['genres'].apply(
    lambda x: [d['name'] for d in x] if isinstance(x, list) else []
)

print(movies[['title', 'genres']].head())


                         title                        genres
0                    Toy Story   [Animation, Comedy, Family]
1                      Jumanji  [Adventure, Fantasy, Family]
2             Grumpier Old Men             [Romance, Comedy]
3            Waiting to Exhale      [Comedy, Drama, Romance]
4  Father of the Bride Part II                      [Comedy]


In [8]:
movies['genres_str'] = movies['genres'].apply(lambda x: " ".join(x) + " ")


In [10]:
import ast

# Fill NaNs first
movies['genres'] = movies['genres'].fillna('[]')

# Convert strings to list if needed
def parse_genres(x):
    if isinstance(x, str):
        return ast.literal_eval(x)
    elif isinstance(x, list):
        return x
    else:
        return []

movies['genres'] = movies['genres'].apply(parse_genres)

# Extract just the genre names (if it's a list of dicts)
movies['genres'] = movies['genres'].apply(
    lambda x: [d['name'] for d in x] if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict) else x
)

# Combine into single string
movies['genres_str'] = movies['genres'].apply(lambda x: " ".join(x))



In [3]:
import pandas as pd

import ast

# Load movies_metadata.csv first
movies = pd.read_csv("movies_metadata.csv", low_memory=False)

# Parse the genres column
movies['genres'] = movies['genres'].fillna('[]').apply(ast.literal_eval)
movies['genres'] = movies['genres'].apply(
    lambda x: [d['name'] for d in x] if isinstance(x, list) else []
)

# Optional: create a string version of genres for text-based analysis
movies['genres_str'] = movies['genres'].apply(lambda x: " ".join(x))


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [5]:
movies.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
genres_str                   0
dtype: int64

In [6]:
movies = movies.iloc[movies['overview'].dropna().index]


In [7]:
movies = movies.fillna(' ')

In [9]:
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')


In [10]:
movies = movies.sort_values(by=['release_date'], ascending=False)

In [11]:
movies = movies.fillna(' ')
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
movies = movies.sort_values(by=['release_date'], ascending=False)


In [12]:
movies.dropna(inplace=True)

In [15]:
import pandas as pd
import ast

# 1) Load the Kaggle file (same folder as your notebook).
#    If your CSVs are inside a subfolder named "data", change to "data/movies_metadata.csv".
movies = pd.read_csv("movies_metadata.csv", low_memory=False)

# 2) Safe parser for the 'genres' column (handles strings like "[{'id': 28, 'name': 'Action'}, ...]").
def parse_genres(val):
    if pd.isna(val):
        return []
    if isinstance(val, list):
        # already parsed
        return [d.get('name', '') for d in val if isinstance(d, dict)]
    if isinstance(val, str):
        try:
            data = ast.literal_eval(val)
            if isinstance(data, list):
                return [d.get('name', '') for d in data if isinstance(d, dict)]
        except Exception:
            return []
    return []

movies["genres_list"] = movies["genres"].apply(parse_genres)
# Optional: squash spaces in genre names so they behave like tags (e.g., "Science Fiction" -> "ScienceFiction")
movies["genres_str"] = movies["genres_list"].apply(lambda lst: " ".join(g.replace(" ", "") for g in lst))

# 3) Ensure text columns exist, fill NaNs with empty strings
for col in ["overview", "tagline", "title"]:
    if col not in movies.columns:
        movies[col] = ""  # create empty if missing (rare)
    movies[col] = movies[col].fillna("").astype(str)

# 4) Build the Tags column (Kaggle version)
movies["Tags"] = (
    movies["genres_str"] + " " +
    movies["overview"] + " " +
    movies["tagline"] + " " +
    movies["title"]
).str.strip()

# 5) Dates: Kaggle uses 'release_date' (lowercase). Coerce bad/missing values.
if "release_date" in movies.columns:
    movies["release_date"] = pd.to_datetime(movies["release_date"], errors="coerce")
    movies = movies.sort_values("release_date", ascending=False)

# 6) Quick sanity check
print(movies[["title", "genres_list", "Tags"]].head(3).to_string(index=False))



                     title                                   genres_list                                                                                                                                                                                                                                                                                                                                                                            Tags
                  Avatar 2 [Action, Adventure, Fantasy, Science Fiction]                                                                                                                                                                                                                                                                                                    Action Adventure Fantasy ScienceFiction A sequel to Avatar (2009).  Avatar 2
The Other Side of the Wind                               [Comedy, Drama] Comedy Drama Orson Welles' unfinished masterp

In [16]:
movies['Tags'] = movies['Tags'].str.lower()

In [17]:
movies.reset_index(inplace=True)

In [18]:
#work with first 10000 movies due to memory space
movies = movies[:10000]

In [19]:
def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)

    def get_wordnet_pos(tag):
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag[0].upper(), wordnet.NOUN)

    return " ".join([lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags])


In [20]:
print(movies.columns)

Index(['index', 'adult', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'genres_list', 'genres_str', 'Tags'],
      dtype='object')


In [21]:
movies.columns = movies.columns.str.lower().str.strip()
print(movies.columns)


Index(['index', 'adult', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'genres_list', 'genres_str', 'tags'],
      dtype='object')


In [25]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
movies['tags_'] = movies['overview'].astype(str).fillna("").apply(lemmatize_sentence)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer  

vectorizer = TfidfVectorizer(max_features=7000, stop_words='english', lowercase=True)
 

In [27]:
vectors = vectorizer.fit_transform(movies['tags']).toarray()

In [28]:
# Step 1: Import everything you need
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 2: Vectorize your movie tags column
vectorizer = CountVectorizer(max_features=5000, stop_words='english')
vectors = vectorizer.fit_transform(movies['tags']).toarray()  # now 'vectors' exists

# Step 3: Build similarity matrix
similarity = cosine_similarity(vectors)

print("Vectors shape:", vectors.shape)
print("Similarity shape:", similarity.shape)


Vectors shape: (10000, 5000)
Similarity shape: (10000, 10000)


In [29]:
similarity = cosine_similarity(vectors) 

In [30]:
similarity.shape

(10000, 10000)

In [34]:
def recommendation(title, data): 
    try:
        movie_index = data[data['TITLE'] == title].index[0] 
    except:
        return "Movie not currently in the database"
    distances = similarity[movie_index] 
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:10]
    for i in movies_list:
        print(data.iloc[i[0]].TITLE) 

In [33]:
movies['original_title'][:100].values


array(['Avatar 2', 'The Other Side of the Wind', 'Bad Boys for Life',
       'Mary Shelley', 'Mobile Homes', 'Iron Sky: The Coming Race',
       'Sly Cooper', "The King's Daughter",
       'How to Talk to Girls at Parties', 'Pitch Perfect 3', 'Machines',
       'Sweet Virginia', 'Justice League', '78/52',
       'Call Me by Your Name', 'Thor: Ragnarok', 'Dina', 'Beyond Skyline',
       'Resurrecting Hassan', 'Leatherface', 'Porto', "God's Own Country",
       'Science Fiction Volume One: The Osiris Child',
       'The Trip to Spain', 'LEGO DC Super Hero Girls: Brain Drain',
       'Patti Cake$', 'What Happened to Monday', 'Ingrid Goes West',
       'Good Time', 'The Glass Castle', 'A Gray State', 'Kidnap',
       'Chronically Metropolitan', 'Columbus', 'The Dark Tower',
       'London Town', 'Wind River', 'Hostages',
       'Cop and a Half: New Recruit', 'S.W.A.T.: Under Siege',
       '東京喰種 トーキョーグール', 'Detroit', 'The Emoji Movie',
       'An Inconvenient Sequel: Truth to Power',
     

In [35]:
def recommendation(title, data, similarity=similarity):
    try:
        # find index of the movie
        movie_index = data[data['original_title'].str.lower() == title.lower()].index[0]
    except IndexError:
        return "Movie not currently in the database"
    
    # get similarity scores
    distances = list(enumerate(similarity[movie_index]))
    distances = sorted(distances, key=lambda x: x[1], reverse=True)[1:6]
    
    # return top 5 similar movies
    return data['original_title'].iloc[[i[0] for i in distances]].tolist()


In [57]:
recommendation("Jumanji", data=movies)

['The Night Flier',
 'Una pura formalità',
 'Judge Dredd',
 'The Mosquito Coast',
 'My Cousin Vinny']

In [58]:
recommendation("Copycat", data=movies)

['H.O.T.S.',
 "All The Queen's Men",
 'Squeeze',
 'King Creole',
 'La vieille qui marchait dans la mer']

In [39]:
movies.to_csv('data.csv', index=False)

In [40]:
import pickle
pickle.dump(similarity, open('similarity.pkl','wb'))

In [41]:
import nltk
nltk.data.path.append(r"C:\Users\kazia\AppData\Roaming\nltk_data")



In [41]:
import os
os.listdir("data")



['credits.csv',
 'keywords.csv',
 'links.csv',
 'links_small.csv',
 'movies_metadata.csv',
 'ratings.csv',
 'ratings_small.csv']

In [42]:
import pandas as pd

# Load the available CSV files
movies = pd.read_csv("data/movies_metadata.csv", low_memory=False)
credits = pd.read_csv("data/credits.csv")
keywords = pd.read_csv("data/keywords.csv")
ratings = pd.read_csv("data/ratings.csv")

print(movies.shape, credits.shape, keywords.shape, ratings.shape)

(45466, 24) (45476, 3) (46419, 2) (26024289, 4)


In [44]:
import pandas as pd

# load the movie metadata
movies = pd.read_csv("data/movies_metadata.csv", low_memory=False)
print(movies.shape)
movies.head()


(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [45]:
ratings = pd.read_csv("data/ratings.csv")
print(ratings.shape)
ratings.head()


(26024289, 4)


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [46]:
keywords = pd.read_csv("data/keywords.csv")
print(keywords.shape)
keywords.head()


(46419, 2)


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [43]:
import os
os.listdir("data")


['credits.csv',
 'keywords.csv',
 'links.csv',
 'links_small.csv',
 'movies_metadata.csv',
 'ratings.csv',
 'ratings_small.csv']

In [47]:
import pandas as pd

# Load metadata files
movies = pd.read_csv("data/movies_metadata.csv", low_memory=False)
credits = pd.read_csv("data/credits.csv")
keywords = pd.read_csv("data/keywords.csv")
links = pd.read_csv("data/links.csv")

# Load ratings
ratings = pd.read_csv("data/ratings.csv")

print("Movies:", movies.shape)
print("Credits:", credits.shape)
print("Keywords:", keywords.shape)
print("Ratings:", ratings.shape)
print("Links:", links.shape)


Movies: (45466, 24)
Credits: (45476, 3)
Keywords: (46419, 2)
Ratings: (26024289, 4)


In [48]:
import os
print(os.getcwd())


C:\Users\kazia\Downloads\Netflix-movie-recommender


In [53]:
import pandas as pd
df = pd.read_csv("movies_metadata.csv", low_memory=False)
print(df.shape)




(45466, 24)


In [56]:
import os
import pandas as pd

BASE_DIR = os.getcwd()   # your project folder

# Define file paths
movies_file   = os.path.join(BASE_DIR, "movies_metadata.csv")
ratings_file  = os.path.join(BASE_DIR, "ratings.csv")
credits_file  = os.path.join(BASE_DIR, "credits.csv")
keywords_file = os.path.join(BASE_DIR, "keywords.csv")
links_file    = os.path.join(BASE_DIR, "links.csv")

# Load datasets
movies   = pd.read_csv(movies_file, low_memory=False)
ratings  = pd.read_csv(ratings_file)
credits  = pd.read_csv(credits_file)
keywords = pd.read_csv(keywords_file)
links    = pd.read_csv(links_file)

# Sanity check
print("Movies:", movies.shape)
print("Ratings:", ratings.shape)
print("Credits:", credits.shape)
print("Keywords:", keywords.shape)
print("links:", links.shape)


Movies: (45466, 24)
Ratings: (26024289, 4)
Credits: (45476, 3)
Keywords: (46419, 2)
links: (45843, 3)


In [54]:
links_file = os.path.join(BASE_DIR, "links.csv")
links = pd.read_csv(links_file)

print("Links:", links.shape)
print(links.head())


Links: (45843, 3)
   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0
